# 🧱 Parent-Child Retrieval e Persistenza con Postgres in LangChain

## 🧭 Obiettivo

Bilanciare **specificità** e **contesto** nella fase di retrieval dividendo i documenti in:

* **Chunk piccoli (child)** → embedding precisi per il recupero
* **Chunk grandi (parent)** → contesto utile nella risposta

---

## 📐 Problema iniziale

* Chunk troppo **piccoli** = embedding accurati, ma poco contesto
* Chunk troppo **grandi** = contesto completo, ma embedding vaghi

> ❗ Serve un modo per **recuperare documenti granulari**, ma **rispondere con documenti più grandi**

---

## 🔁 Soluzione: Parent-Child Retriever

LangChain fornisce una struttura in cui:

* I **chunk piccoli** vengono inseriti nel **vector store**
* Ogni chunk mantiene un riferimento (`doc_id`) al suo **documento padre** nel **docstore**
* In fase di retrieval:

  1. Si cercano i chunk più simili
  2. Si leggono i `doc_id` associati
  3. Si caricano i documenti completi (parent) dal `docstore`


![alt](../images/parent-child.png)

---

## 🛠 Composizione del sistema

```txt
          +----------------+         +-------------------+
          |   VectorStore  |         |     DocStore      |
          | (chunk piccoli)|         | (documenti grandi)|
          +----------------+         +-------------------+
                  |                           |
      [retrieval con embedding]      [lookup da doc_id]
                  |                           |
            +-------------------------------+
            |   Parent Documents retrieved   |
            +-------------------------------+
```

### Preparazione

In [1]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders.directory import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os
load_dotenv()

loader = DirectoryLoader("./data", glob="**/*.txt")

docs = loader.load()

model = ChatOpenAI()

# nel vectorstore memorizziamo i piccoli chunks che poi andranno retrievati
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OpenAIEmbeddings()
)

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
C:\Users\felip\AppData\Local\Temp\ipykernel_15488\3917626230.py:17: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


---

## 💾 LangChain In-Memory DocStore

Recuperati i piccoli chunks dal vectorstore si cercano gli IDs dei genitori con i quale recuperiamo apppunto i documenti più grandi (genitori) dal InMemoryStore

* Classe `InMemoryDocstore`
* Utile per **prototipazione**, ma **non persistente**
* Se riavvii il server, **perdi tutti i documenti**

### Uso base

In [2]:

from langchain.storage import InMemoryStore
from langchain.retrievers import ParentDocumentRetriever



---

## 🧪 Differenza tra Splitter Genitore e Figlio

* **Parent Splitter**: divide i documenti in **blocchi medi**, da salvare nel `docstore`
* **Child Splitter**: divide ogni blocco genitore in **pezzi piccoli**, da salvare nel `vectorstore`

> Lo splitter padre è **opzionale**, lo splitter figlio è **obbligatorio**


In [3]:
docstore = InMemoryStore()
child_splitter = RecursiveCharacterTextSplitter(chunk_size=250)

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=600)

# con ParentDocumentRetriever() splittiamo i documenti genitori prima 
# quindi la dimensione dei chunks deve essere più grande della dimensione dei chunks figli 
# supponiamo di avere tre documenti .txt, e che poi lo splitter genitore divida i documenti in 10 chunks
# e poi uno splitter figlio che crea chunks ancora più piccoli, ad esempio in 30 chunks

# facciamo una prova prima senza il parent_splitter

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore, 
    docstore=docstore,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)

retriever.add_documents(docs, ids=None)

In [4]:
# usiamo il metodo yield_keys() per ricavare dal docstore le chiavi 
# e vedere quanti documenti abbiamo al suo interno

len(list(docstore.yield_keys()))

22

Quindi abbiamo 3 documenti .txt, 22 chuks più grandi nel docstore, e ancora di più chunks più piccoli nel vectorstore su cui eseguiamo le nostre query.

In [5]:
retriever.invoke("who is the owner?")

[Document(metadata={'source': 'data\\restaurant.txt'}, page_content="One evening, as the sun cast a golden glow over the city, a renowned food critic, Elena Rossi, stepped into Chef Amico. Her mission was to uncover the secret behind the restaurant's growing fame. She was greeted by Amico himself, whose eyes sparkled with the joy of a man who loved his work."),
 Document(metadata={'source': 'data\\founder.txt'}, page_content='As he grew, so did his desire to explore beyond the shores of Sicily. Venturing through Italy, Amico worked alongside renowned chefs, each teaching him a new facet of Italian cuisine. From the rolling hills of Tuscany to the romantic canals of Venice, he absorbed the diverse regional flavors, techniques, and traditions that would later influence his unique culinary style.\n\nCreating Chef Amico’s Restaurant'),
 Document(metadata={'source': 'data\\founder.txt'}, page_content='Creating Chef Amico’s Restaurant\n\nReturning to Palermo with a vision, Amico opened the d

---

## 🚫 Limiti della memoria volatile

* ❌ `InMemoryStore` non è adatto a produzione
* ✅ Serve un sistema **persistente**, ad esempio **PostgreSQL**

---

## 🧱 Creare un DocStore personalizzato con PostgreSQL

### 1. 🔖 Chiavi e metadati

* Ogni documento ha una `doc_id` nei metadati
* LangChain si aspetta che il `docstore` gestisca una mappa chiave → valore

### 2. 📦 Serializzazione

* I documenti vengono **serializzati** in JSON per essere salvati
* Alla lettura, il JSON viene **deserializzato** in `Document`

---

## 🧾 Struttura della tabella Postgres

```sql
CREATE TABLE documents (
    key TEXT PRIMARY KEY,
    value JSONB NOT NULL
);
```

* `key` → identificatore univoco del documento (`doc_id`)
* `value` → JSON serializzato del documento LangChain

---

## 📌 Implementare il DocStore personalizzato

### Requisiti minimi (ereditando da `BaseStore`):

* `mget(keys: List[str]) → Dict[str, Any]`
* `mset(kv_pairs: Dict[str, Any]) → None`
* `yield_keys() → Iterator[str]`

Optional async versions:

* `amget`, `amset`, `amdelete`

---

In [6]:
# creiamo il nostro modello di documento

from pydantic import BaseModel, Field
from typing import Optional


# proprtio come i Document di Langchain ma con io campo key in più 
class DocumentModel(BaseModel):
    key: Optional[str] = Field(None)
    page_content: Optional[str] = Field(None)
    metadata: dict = Field(default_factory=dict)

- la tabella consiste di sole due colonne key e value
- la colonna key è una stringa ed la primary key della tabella 
- la colonna value è il JSON Binary 
- abbiamo una lista di Document che saranno convertiti in un oggetto JSON dato che non 
- è possibile memorizzare direttamente le classi Python in un database, tale processo è chimaato serializzazione 
- qundo recuperiamo i dati dal database convertiamo l'oggetto JSON in una classe Document, quindi eseguiamo la deserializzazione

In [8]:
# creiamo la nostra tabella per sql

from sqlalchemy import Column, String, create_engine
from sqlalchemy.orm import declarative_base
from sqlalchemy.dialects.postgresql import JSONB

Base = declarative_base()

class SQLDocument(Base):
    __tablename__ = "docstore"
    key = Column(String, primary_key=True)
    value = Column(JSONB)

    def __repr__(self):
        return f"<SQLDocument(key='{self.key}', value='{self.value}')>"



## 🧪 Ricapitolando il flusso

```text
1. Documenti grandi → splittati in chunk genitori
2. Chunk genitori → salvati nel Postgres docstore
3. Chunk figli → derivati da quelli genitori → salvati nel vector store
4. Retrieval → su embedding dei figli
5. Lookup → doc_id → carica documento completo dal Postgres docstore
```